In [1]:
import pandas as pd
from pymongo import MongoClient

# 1️⃣ Conexión a MongoDB (asegúrate de que el servicio esté corriendo)
client = MongoClient("mongodb://localhost:27017/")

# 2️⃣ Crear base de datos y colección según el examen
db = client["TurismoPeru_2025"]          # Nombre de la base de datos
collection = db["Visitantes"]             # Nombre de la colección

# 3️⃣ Cargar datos limpios desde CSV
df = pd.read_csv("../database/visitantes_clean.csv")

# 4️⃣ Convertir DataFrame a lista de diccionarios (formato JSON para MongoDB)
data = df.to_dict(orient="records")

# 5️⃣ Insertar en MongoDB (primero limpiar colección si ya existe, opcional)
collection.delete_many({})  # Elimina todos los documentos anteriores (útil para pruebas)
collection.insert_many(data)

print("✅ Datos insertados en MongoDB correctamente.")
print(f"📁 Base de datos: TurismoPeru_2025 | Colección: Visitantes | Registros: {len(data)}")

# ==============================================================================
# 🔍 CONSULTAS DE EJEMPLO (contexto turístico)
# ==============================================================================

## ✅ A. Mostrar todos los documentos (solo los primeros 3 para no saturar)
print("\n🔍 A. Primeros 3 documentos completos:")
for doc in collection.find().limit(3):
    print(doc)

## ✅ B. Mostrar solo algunos campos: región y destino
print("\n🔍 B. Región y destino de los visitantes:")
for doc in collection.find({}, {"_id": 0, "region": 1, "destino": 1}).limit(5):
    print(doc)

## ✅ C. Consultar visitantes extranjeros (país de origen ≠ Perú)
print("\n🔍 C. Visitantes extranjeros (primeros 5):")
consulta_extranjeros = {"pais_origen": {"$ne": "Perú"}}
for doc in collection.find(consulta_extranjeros, {"_id": 0, "pais_origen": 1, "region": 1, "gasto_total": 1}).limit(5):
    print(doc)

## ✅ D. Filtrar por gasto alto (gasto_total > 3000)
print("\n🔍 D. Visitantes con gasto mayor a 3000 USD:")
consulta_gasto_alto = {"gasto_total": {"$gt": 3000}}
for doc in collection.find(consulta_gasto_alto, {"_id": 0, "id_visitante": 1, "gasto_total": 1, "region": 1}).limit(5):
    print(doc)

## ✅ E. Contar total de registros
total = collection.count_documents({})
print(f"\n🔍 E. Total de visitantes en la base: {total}")

## ✅ F. Visitantes a una región específica (ej: Cusco)
print("\n🔍 F. Visitantes a Cusco (primeros 3):")
for doc in collection.find({"region": "Cusco"}, {"_id": 0, "id_visitante": 1, "destino": 1, "gasto_total": 1}).limit(3):
    print(doc)

## ✅ G. Visitantes que usaron avión como transporte
print("\n🔍 G. Visitantes que viajaron en avión:")
for doc in collection.find({"medio_transporte": "Avión"}, {"_id": 0, "pais_origen": 1, "region": 1}).limit(5):
    print(doc)

✅ Datos insertados en MongoDB correctamente.
📁 Base de datos: TurismoPeru_2025 | Colección: Visitantes | Registros: 2834

🔍 A. Primeros 3 documentos completos:
{'_id': ObjectId('68f24f1bc9383466dad249fa'), 'id_visitante': 'V0001', 'pais_origen': 'Brasil', 'edad': 59, 'genero': 'F', 'destino': 'Río Amazonas', 'region': 'Iquitos', 'fecha_visita': '2025-01-10', 'dias_estadia': 12, 'gasto_total': 3474.65, 'medio_transporte': 'Desconocido'}
{'_id': ObjectId('68f24f1bc9383466dad249fb'), 'id_visitante': 'V0002', 'pais_origen': 'Colombia', 'edad': 17, 'genero': 'M', 'destino': 'Islas Flotantes Uros', 'region': 'Puno', 'fecha_visita': '2025-09-15', 'dias_estadia': 11, 'gasto_total': 890.28, 'medio_transporte': 'Desconocido'}
{'_id': ObjectId('68f24f1bc9383466dad249fc'), 'id_visitante': 'V0003', 'pais_origen': 'Argentina', 'edad': 25, 'genero': 'Otro', 'destino': 'Huaca Pucllana', 'region': 'Lima', 'fecha_visita': '2025-05-01', 'dias_estadia': 3, 'gasto_total': 3565.26, 'medio_transporte': 'Auto